In [ ]:
import numpy as np

def convertToOneHot(vector, num_classes=None):
    """
    Converts an input 1-D vector of integers into an output
    2-D array of one-hot vectors, where an i'th input value
    of j will set a '1' in the i'th row, j'th column of the
    output array.

    Example:
        v = np.array((1, 0, 4))
        one_hot_v = convertToOneHot(v)
        print one_hot_v

        [[0 1 0 0 0]
         [1 0 0 0 0]
         [0 0 0 0 1]]
    """

    assert isinstance(vector, np.ndarray)
    assert len(vector) > 0

    if num_classes is None:
        num_classes = np.max(vector)+1
    else:
        assert num_classes > 0
        assert num_classes >= np.max(vector)

    result = np.zeros(shape=(len(vector), num_classes))
    result[np.arange(len(vector)), vector] = 1
    return result.astype(int)

In [ ]:
with open("/data/xuht/ChineseSTSListCorpus/bert/bert_cnn/vocab_label.pik", "rb") as frobj:
    import _pickle as pkl
    d = pkl.load(frobj)

In [ ]:
with open("/data/xuht/finical_unigram.vocab", "r") as frobj:
    lines = frobj.read().splitlines()

In [ ]:
!pip install sentencepiece

In [ ]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load("/data/xuht/finical_unigram.model")

In [ ]:
sent = "记者注意到,以目前杉杉股份的生产经营情况来看,虽然公司正极材料的盈利能力不错,但公司负极材料的盈利能力一般"
l = sp.EncodeAsIds(sent)
k = sp.EncodeAsPieces(sent)
print(len(sent), len(l), len(k))
print(sent, l, k)

In [ ]:
sent = "wahh1 dfsgv"
l = sp.EncodeAsIds(sent)
print(len(sent), len(l))
print(sent, l)

In [ ]:
with open("/data/xuht/finical_unigram.vocab") as frobj:
    lines = frobj.read().splitlines()
sp.PieceToId("[cls]")

In [ ]:
lines[0:10]

In [ ]:
import tensorflow as tf
def read_my_file_format(filename_queue):
  reader = tf.SomeReader()
  key, record_string = reader.read(filename_queue)
  example, label = tf.some_decoder(record_string)
  processed_example = some_processing(example)
  return processed_example, label

def input_pipeline(filenames, batch_size, num_epochs=None):
  filename_queue = tf.train.string_input_producer(
      filenames, num_epochs=num_epochs, shuffle=True)
  example, label = read_my_file_format(filename_queue)
  # min_after_dequeue defines how big a buffer we will randomly sample
  #   from -- bigger means better shuffling but slower start up and more
  #   memory used.
  # capacity must be larger than min_after_dequeue and the amount larger
  #   determines the maximum we will prefetch.  Recommendation:
  #   min_after_dequeue + (num_threads + a small safety margin) * batch_size
  min_after_dequeue = 10000
  capacity = min_after_dequeue + 3 * batch_size
  example_batch, label_batch = tf.train.shuffle_batch(
      [example, label], batch_size=batch_size, capacity=capacity,
      min_after_dequeue=min_after_dequeue)
  return example_batch, label_batch

In [ ]:
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = ""
filename = "/data/xuht/eventy_detection/bert/guangfa/bert/pretrain_finicial_new.tfrecords"
with tf.Graph().as_default():
    sess = tf.Session()

    max_seq_length = 128
    max_predictions_per_seq = 128
    name_to_features = {
        "input_ids":
            tf.FixedLenFeature([max_seq_length], tf.int64),
        "input_mask":
            tf.FixedLenFeature([max_seq_length], tf.int64),
        "segment_ids":
            tf.FixedLenFeature([max_seq_length], tf.int64),
        "masked_lm_positions":
            tf.FixedLenFeature([max_predictions_per_seq], tf.int64),
        "masked_lm_ids":
            tf.FixedLenFeature([max_predictions_per_seq], tf.int64),
        "masked_lm_weights":
            tf.FixedLenFeature([max_predictions_per_seq], tf.float32),
        "next_sentence_labels":
            tf.FixedLenFeature([1], tf.int64),
    }
    
    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)
        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example

    def tfrecord_():
        input_files = tf.gfile.Glob(filename)
        d = tf.data.Dataset.from_tensor_slices(tf.constant(input_files))
        print(len(input_files), "============")
        d = d.repeat()
        d = d.shuffle(buffer_size=len(input_files))

        # `cycle_length` is the number of parallel files that get read.
        cycle_length = min(4, len(input_files))

        # `sloppy` mode means that the interleaving is not exact. This adds
        # even more randomness to the training pipeline.
        d = d.apply(
                tf.contrib.data.parallel_interleave(
                        tf.data.TFRecordDataset,
                        sloppy=True,
                        cycle_length=cycle_length))
        d = d.shuffle(buffer_size=100)
        d = d.make_one_shot_iterator()

        return d.get_next()
    
    def tfrecord_train_input_fn():
        tfrecord_dataset = tf.data.TFRecordDataset(filename)
        tfrecord_dataset = tfrecord_dataset.map(lambda   x:_decode_record(x, name_to_features)).shuffle(True).batch(20)
        tfrecord_dataset = tfrecord_dataset.repeat(1)
        tfrecord_iterator = tfrecord_dataset.make_one_shot_iterator()

        return tfrecord_iterator.get_next()
    
    att = tf.placeholder(tf.int32,[None, None]) 
    
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)
    
    a = tfrecord_train_input_fn()
    b = tfrecord_()
    
    for l in range(1):
        r = sess.run(b)
        print(r)

#     filename_queue = tf.train.string_input_producer([filename], num_epochs=1)
#     reader = tf.TFRecordReader()
#     _, serialized_example = reader.read(filename_queue)
#     output = _decode_record(serialized_example, name_to_features)
#     print(output)
#     example = tf.train.shuffle_batch([output], batch_size=1, capacity=2, num_threads=1, min_after_dequeue=1)
#     threads = tf.train.start_queue_runners(coord=coord, sess = sess)
#     for batch_index in range(1):
#         example = sess.run(example)
#         print(example)

In [ ]:
import jieba
jieba.add_word("腾讯游戏", 100)
jieba.add_word("腾讯", 100)
list(jieba.cut("腾讯游戏业绩下滑"))

In [ ]:
jieba.add_word("<target>", 100)
list(jieba.cut("<target>dskgfj"))



In [6]:
with open("/data/xuht/Chinese_w2v/Tencent_AILab_ChineseEmbedding/vocab.txt", "r") as frobj:
    general_vocab = frobj.read().splitlines()
with open("/data/xuht/general_dict/Company-Names-Corpus.txt", "r") as frobj:
    company_name = frobj.read().splitlines()[3:]
with open("/data/xuht/general_dict/Chinese_Names_Corpus.txt", "r") as frobj:
    person_name = frobj.read().splitlines()[3:]

In [9]:
import jieba
for word in general_vocab+company_name+person_name:
    jieba.add_word(word, 10000)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.841 seconds.
Prefix dict has been built succesfully.


In [12]:
import pandas as pd
data = pd.read_excel("/data/xuht/eventy_detection/data_1114.xlsx")

In [15]:
sent_dict = {}
for index in range(data.shape[0]):
    sent = data.loc[index]["句子内容"]
    if sent in data:
        sent_dict[sent].append(data.loc[index]["预期事件"])
    else:
        sent_dict[sent] = [data.loc[index]["预期事件"]]
single_event = {}
for key in sent_dict:
    sent_dict[key] = list(set(sent_dict[key]))
    if len(sent_dict[key]) == 1:
        single_event[key] = sent_dict[key]

In [17]:
event_dict = {}
for key in single_event:
    if single_event[key][0] in event_dict:
        event_dict[single_event[key][0]].append(key)
    else:
        event_dict[single_event[key][0]] = [key]

event_sent = {}
for key in event_dict:
    for sent in event_dict[key]:
        word_lst = list(jieba.cut(sent))
        

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse.csr import csr_matrix #need this if you want to save tfidf_matrix

feature_names = tf.get_feature_names()

tf = TfidfVectorizer(input=, analyzer='word', ngram_range=(1,6),
                     min_df = 0, stop_words = 'english', sublinear_tf=True)
tfidf_matrix =  tf.fit_transform(corpus)

75

In [ ]:
# Starting with the CountVectorizer/TfidfTransformer approach...
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

cvec = CountVectorizer(stop_words='english', min_df=1, max_df=.5, ngram_range=(1,3))